In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os  # Trabajar con funciones del SO, rutas, variables de entorno
import requests  # Peticiones HTTP a webs o APIs
from dotenv import load_dotenv  # Cargar variables desde un archivo .env

# Selenium: para automatizar y controlar el navegador
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  # Configuración de Chrome
from selenium.webdriver.common.by import By  # Localizar elementos (por tag, id, clase, etc.)
from selenium.webdriver.support.ui import WebDriverWait  # Esperas explícitas
from selenium.webdriver.support import expected_conditions as EC  # Condiciones de espera
from selenium.webdriver.edge.options import Options as EdgeOptions

# BeautifulSoup: para parsear y limpiar el HTML
from bs4 import BeautifulSoup

# IPython: para mostrar contenido en formato Markdown (solo si usas Jupyter)
from IPython.display import Markdown, display

# OpenAI API
from openai import OpenAI


In [4]:
# Cargamos las variables del fichero .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No se encontró ninguna clave API: diríjase al cuaderno de resolución de problemas en esta carpeta para identificarla y solucionarla.")
elif not api_key.startswith("sk-proj-"):
    print("Se encontró una clave API, pero no inicia sk-proj-; verifique que esté usando la clave correcta; consulte el cuaderno de resolución de problemas")
elif api_key.strip() != api_key:
    print("Se encontró una clave API, pero parece que puede tener espacios o caracteres de tabulación al principio o al final; elimínelos; consulte el cuaderno de resolución de problemas")
else:
    print("¡Se encontró la clave API y hasta ahora parece buena!")


¡Se encontró la clave API y hasta ahora parece buena!


In [5]:
openai = OpenAI()#Creamos el Objeto Cliente con OpenAI() y lo guardamos acá para dps usarlo

# Si esto no funciona, prueba con el menú Kernel >> Reiniciar Kernel y borrar las salidas de todas las celdas, luego ejecuta las celdas desde la parte superior de este cuaderno hacia abajo.
# Si TODAVÍA no funciona (¡qué horror!), consulta el cuaderno de resolución de problemas o prueba la siguiente línea:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

In [6]:
# Una clase para representar una página web
# Si no estás familiarizado con las clases, consulta el cuaderno "Python intermedio"

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado
    """

    #1.Config del constructor
    def __init__(self, url):
        self.url = url
        self.title = ""
        self.text = ""

    #2.Config de el motor en modo headless
    def load(self):
        options = EdgeOptions()
        #options.add_argument("--headless=new")#Para q el motor se ejecute en 2do plano
        #options.add_argument("--disable-gpu")

        driver = webdriver.Edge(options=options)


        try:
            #3.Abrimos la url mediante el get
            driver.get(self.url)

            #4.Esperar a que cargue el body 
            WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.TAG_NAME,"body"))
            )
            """
               -Creamos un esperador q intentará encontrar algo en la página durante hasta 10s:si lo encuentra al instante sigue de inmediato sin esperar ese tiempo y si no lo encuentra en ese tiempo lanza un error->TimeoutException
               -.until() es un "esperá hasta que la condición se cumpla"
               -EC.presence_of....->es la condicion en gnal
               -EC->expected conditions/condiciones esperadas
               ->By.Tag_Name,"body"->indica que queremos encontrar un elemento cuyo tag HTML sea un body
               Resumen de ese fragmento: Básicamente pausa la ejecucion hasta que la página tenga un <body> en el DOM
            """

            #5.Obtener HTML y limpiar
            soup = BeautifulSoup(driver.page_source, "html.parser")
            """
                driver.page_source->le pide a selenium todo el code HTML
                BeautifulSoup(..., "html.parser")->toma ese html en bruto y lo pasa a una estructura q se puede recorrer y manipular facilmente con python(esto sería objeto soup)
                Soup-> al final soup te deja modif o eliminar etiquetas y extraer txt de forma muy facil
            """
        
            #Hay título a extraer sí o no? 
            self.title = soup.title.string if soup.title else "No tiene título"

            #Eliminamos lo innecesario
            for tag in soup(["script","style","img"]):
                tag.decompose()
            #Extraemos todo el txt plano del html ya limpio
            self.text = soup.get_text(separator = "\n",strip = True)
        
        finally:
            #Finalmente sin importar si hubo error o no cerramos el navegador para liberar recursos
            driver.quit()

In [7]:
site = Website("https://openai.com/es-419/index/introducing-gpt-5/")
site.load()

print(site.title)
print(site.text) 

Introducing GPT-5 | OpenAI
Introducing GPT-5 | OpenAI
Pasar al contenido principal
Iniciar sesión
Cambiar a
ChatGPT
(se abre en una nueva ventana)
Sora
(se abre en una nueva ventana)
API Platform
(se abre en una nueva ventana)
Inicio
Índice de investigaciones
Descripción general de investigaciones
Residencia de investigaciones
Últimos avances
GPT-5
OpenAI o3 y o4-mini
GPT-4.5
OpenAI o1
GPT-4o
Sora
Investigación
Volver al menú principal
Índice de investigaciones
Descripción general de investigaciones
Residencia de investigaciones
Últimos avances
GPT-5
OpenAI o3 y o4-mini
GPT-4.5
OpenAI o1
GPT-4o
Sora
Seguridad
Volver al menú principal
Enfoque de seguridad
Seguridad y privacidad
Para empresas
Volver al menú principal
Descripción general de los servicios para empresas
Soluciones
ChatGPT Pricing
API Pricing
Comunícate con Ventas
Plataforma API
Volver al menú principal
Descripción general de la plataforma
Precios
Codex
Open Models
Foro de desarrolladores
(se abre en una nueva ventana)
ChatG

In [8]:
def messages_for(website):
    return [
        {"role": "system", "content": "Eres un asistente que resume páginas web."},
        {"role": "user", "content": f"Resume el siguiente contenido:\n\nTítulo: {website.title}\n\nTexto:\n{website.text}"}
    ]

def summarize(url):
    website = Website(url)
    website.load()  # importante: cargar contenido
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [10]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [12]:
display_summary("https://openai.com/es-419/index/introducing-gpt-5/")

OpenAI ha presentado GPT-5, su modelo de inteligencia artificial más avanzado hasta la fecha, que ofrece mejoras significativas en diversos campos como programación, escritura, salud y percepción visual. GPT-5 combina un sistema unificado que permite una rápida respuesta y un modelo de razonamiento profundo, optimizando la interacción según el tipo y complejidad de la consulta.

El modelo destaca en la creación de código, generando aplicaciones web y juegos de forma intuitiva. También se ha mejorado en capacidades de escritura, ayudando a transformar ideas iniciales en textos impactantes y elaborados. En el ámbito de la salud, GPT-5 supera a modelos anteriores al actuar como un socio activo, ofreciendo respuestas más precisas y contextuales.

GPT-5 es menos propenso a generar errores (alucinaciones) y comunica de manera más honesta sus limitaciones. Se ha implementado una nueva forma de entrenamiento en seguridad, asegurando que el modelo proporcione respuestas útiles mientras navega por solicitudes ambivalentes.

Además, se ha introducido una variante llamada GPT-5 Pro para tareas más complejas, obteniendo mejores resultados en evaluación de inteligencia y en casos del mundo real. Los usuarios de ChatGPT pueden acceder a GPT-5, que se convierte en el modelo por defecto, con suscriptores Plus y Pro disfrutando de mayor capacidad de uso.

En resumen, GPT-5 representa un avance significativo en la inteligencia artificial, ofreciendo un modelo más seguro, útil y personalizable para los usuarios.